In [1]:
import pandas as pd
excel_path = 'data/map/namdinh/data.xlsx'
xls = pd.ExcelFile(excel_path)
all_data_with_sheet_name = []

# Iterate through each sheet, add a column for 'varname' to store the sheet name, and then append to the list
for sheet_name in xls.sheet_names:
    df_sheet = pd.read_excel(xls, sheet_name=sheet_name)
    # For each sheet, melt and then add the sheet name as 'varname'
    df_sheet_melted = df_sheet.melt(id_vars=["NAME"], var_name="YEAR", value_name="value")
    df_sheet_melted["varname"] = sheet_name
    all_data_with_sheet_name.append(df_sheet_melted)

# Concatenate all the melted DataFrames with the sheet name included
long_df_with_varname = pd.concat(all_data_with_sheet_name, ignore_index=True)

long_df_with_varname

,NAME,YEAR,value,varname
0,Thị trấn Mỹ Lộc,2012,4859.000000,pop
1,Xã Mỹ Hà,2012,7428.000000,pop
2,Xã Mỹ Tiến,2012,5049.000000,pop
3,Xã Mỹ Thắng,2012,7932.000000,pop
4,Xã Mỹ Trung,2012,4984.000000,pop
...,...,...,...,...
9939,Xã Hải Chính,2021,1.686910,perph
9940,Xã Hải Xuân,2021,3.566591,perph
9941,Xã Hải Châu,2021,0.043328,perph
9942,Xã Hải Triều,2021,2.382486,perph


In [2]:
# Separate the DataFrames based on 'varname' for 'ph' and 'hh' using 'contains'
df_ph = long_df_with_varname[long_df_with_varname['varname'].str.contains('ph')].copy()
df_hh = long_df_with_varname[long_df_with_varname['varname'].str.contains('hh')].copy()

# Perform operations directly on the copy to avoid SettingWithCopyWarning
df_ph['YEAR'] = df_ph['YEAR']
df_hh['YEAR'] = df_hh['YEAR']

# Merge the 'ph' and 'hh' DataFrames on 'NAME' and 'year'
df_merged = pd.merge(df_ph, df_hh, on=['NAME', 'YEAR'], suffixes=('_ph', '_hh'))

# Calculate the new variable 'ph/hh * 100'
df_merged['perph'] = (df_merged['value_ph'] / df_merged['value_hh'])

# Create a DataFrame for the new variable
df_new_var = df_merged[['NAME', 'YEAR', 'perph']].copy()
df_new_var['varname'] = 'perph'

# Ensure the original DataFrame is not just a view to avoid the warning
long_df_with_varname_filtered = long_df_with_varname[~long_df_with_varname['varname'].str.contains('ph|hh')].copy()

# Concatenate the DataFrame with the new variable to the filtered original DataFrame
final_df = pd.concat([long_df_with_varname_filtered, df_new_var[['NAME', 'YEAR', 'varname', 'perph']].rename(columns={'perph': 'value'})], ignore_index=True)

# Note: Ensure to use '.copy()' when slicing DataFrames to avoid the SettingWithCopyWarning when setting new values.


In [3]:
final_df

,NAME,YEAR,value,varname
0,Thị trấn Mỹ Lộc,2012,4859.000000,pop
1,Xã Mỹ Hà,2012,7428.000000,pop
2,Xã Mỹ Tiến,2012,5049.000000,pop
3,Xã Mỹ Thắng,2012,7932.000000,pop
4,Xã Mỹ Trung,2012,4984.000000,pop
...,...,...,...,...
4955,Xã Hải Chính,2021,0.001138,perph
4956,Xã Hải Xuân,2021,0.001610,perph
4957,Xã Hải Châu,2021,0.000019,perph
4958,Xã Hải Triều,2021,0.001534,perph


In [4]:
pivot_df = final_df.pivot_table(index=['NAME', 'YEAR'], columns='varname', values='value').reset_index()

In [5]:
pivot_df

varname,NAME,YEAR,perph,pop
0,Phường Hạ Long,2012,NaN,15568.0
1,Phường Hạ Long,2013,NaN,15761.0
2,Phường Hạ Long,2014,NaN,15948.0
3,Phường Hạ Long,2015,NaN,16059.0
4,Phường Hạ Long,2016,0.008032,14760.0
...,...,...,...,...
2435,Yên Đồng,2017,0.016499,NaN
2436,Yên Đồng,2018,0.012909,NaN
2437,Yên Đồng,2019,0.008869,NaN
2438,Yên Đồng,2020,0.004765,NaN


In [7]:
pivot_df['NAME'] = pivot_df['NAME'].str.replace(' ','').str.replace('Thịtrấn','').str.replace('Phường','')

In [8]:
pivot_df

varname,NAME,YEAR,perph,pop
0,HạLong,2012,NaN,15568.0
1,HạLong,2013,NaN,15761.0
2,HạLong,2014,NaN,15948.0
3,HạLong,2015,NaN,16059.0
4,HạLong,2016,0.008032,14760.0
...,...,...,...,...
2435,YênĐồng,2017,0.016499,NaN
2436,YênĐồng,2018,0.012909,NaN
2437,YênĐồng,2019,0.008869,NaN
2438,YênĐồng,2020,0.004765,NaN
